In [ ]:
# Installing Pathway
!pip install -U pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [4]:
# Importing data
from google.colab import files
uploaded = files.upload()

Saving input_data.csv to input_data (2).csv


In [13]:
# Importing Pathway
import pathway as pw
import datetime

In [14]:
# Parking Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [15]:
# Reading Data
data = pw.io.csv.read(
    "input_data.csv",
    schema=ParkingSchema,
    mode="static",
    # autocommit_duration_ms=10
)

In [16]:
# Adding Hour Column
@pw.udf
def extract_hour(ts: str) -> int:
    return datetime.datetime.fromisoformat(ts).hour

data = data.with_columns(
    Hour=extract_hour(pw.this.Timestamp)
)

In [17]:
# Computing Price
@pw.udf
def compute_price(
    occupancy, capacity, queue_length,
    traffic_condition, is_special_day,
    vehicle_type, hour
):
    base_price = 10.0
    demand_factor = (occupancy + queue_length) / max(capacity, 1)
    if is_special_day:
        demand_factor *= 1.2
    if traffic_condition == "High":
        demand_factor *= 0.9
    if traffic_condition == "Low":
        demand_factor *= 1.1
    if vehicle_type == "Truck":
        demand_factor *= 1.3
    if vehicle_type == "Bike":
        demand_factor *= 0.9
    if 11 <= hour <= 14:
        demand_factor *= 1.2
    if 8 <= hour <= 9:
        demand_factor *= 0.85
    price = base_price + demand_factor * 2.5
    return round(max(price, 0), 2)

In [18]:
# Output
output_table = data.with_columns(
    Price=compute_price(
        pw.this.Occupancy, pw.this.Capacity, pw.this.QueueLength,
        pw.this.TrafficConditionNearby, pw.this.IsSpecialDay,
        pw.this.VehicleType, pw.this.Hour
    )
)

In [19]:
# Print in the console
pw.debug.compute_and_print(output_table)

            | Timestamp           | SystemCodeNumber | Occupancy | Capacity | QueueLength | TrafficConditionNearby | IsSpecialDay | VehicleType | Hour | Price
^MBV9QRR... | 2025-07-05T08:00:00 | LOT_A            | 25        | 100      | 5           | High                   | 1            | Car         | 8    | 10.69
^GC3HNJZ... | 2025-07-05T09:00:00 | LOT_A            | 40        | 100      | 10          | Medium                 | 0            | Truck       | 9    | 11.38
^AEWRKDH... | 2025-07-05T10:00:00 | LOT_B            | 60        | 120      | 2           | Low                    | 0            | Car         | 10   | 11.42
^Y583XVD... | 2025-07-05T11:00:00 | LOT_B            | 90        | 120      | 15          | High                   | 1            | Truck       | 11   | 13.69
^K8RCNG6... | 2025-07-05T12:00:00 | LOT_C            | 20        | 80       | 1           | Medium                 | 0            | Car         | 12   | 10.79
